In [1]:
import pprint
import orcid
from requests import RequestException

In [2]:
api = orcid.PublicAPI()

In [9]:
ringgoldid= '6187'
place = "Oklahoma"

In [ ]:
university = api.search_public('%s %s' % (ringgoldid, place), rows=10)

In [ ]:
university

In [3]:
ringgoldid= '6187'
place='University of Oklahoma'

In [ ]:
results = university.get('orcid-search-results').get('orcid-search-result')

In [ ]:
result = results[0]

In [ ]:
result

In [ ]:
theid = result.get('orcid-profile').get('orcid-identifier').get('path')
print (theid)
fname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('given-names').get('value')
print (fname)
lname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('family-name').get('value')
print (lname)

In [ ]:
fname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('given-names').get('value')
print (fname)

In [ ]:
type(fname)

In [ ]:
for key in result:
    print (key)


In [ ]:
import pprint
pprint.pprint(result.keys())

In [ ]:
r = (result['orcid-profile'])

In [ ]:
pprint.pprint(r.keys())

In [ ]:
theid = '0000-0001-5544-0976'
summary = api.read_record_public(theid, 'activities')

In [ ]:
summary

In [ ]:
employ = summary.get('employments').get('employment-summary')

In [ ]:
pprint.pprint(employ)

In [ ]:
id = {}
d = {}
id.setdefault(theid,[]).append(fname + ' ' + lname)
d.setdefault('name',[]).append(id)

In [ ]:
import pprint
import orcid
import time
from requests import RequestException

def search_persons(ringgoldid, place):
    api = orcid.PublicAPI()
    persons = {}
    counter = 1
    row = 20
    total_records = api.search_public('%s %s' % (ringgoldid, place))['orcid-search-results']['num-found']
    print ('The total number of entries matching', ringgoldid, 'and', place, 'is', total_records)
    total_records = 20

    # define log entries
    tmpl = '%s\t%s\t%s\n'

    for step in range(0, total_records, row):
        try:
            university = api.search_public('%s %s' % (ringgoldid, place), start=str(step), rows=row)
            for result in university.get('orcid-search-results').get('orcid-search-result'):
                orcid_id = result.get('orcid-profile').get('orcid-identifier').get('path')
              #  request_log.write(tmpl % (datetime.isoformat(datetime.now()), counter, orcid_id))
                
                try:
                    fname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('given-names').get('value')
                except:
                    fname = ''
                try:
                    lname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('family-name').get('value')
                except:
                    lname = ''
                
                try:
                    summary = api.read_record_public(orcid_id, 'activities')
                    try:
                        for employment in summary.get('employments').get('employment-summary'):
                            if employment.get('organization').get('disambiguated-organization').get('disambiguated-organization-identifier') == ringgoldid:
                                if not employment.get('end-date'):
                                    persons.setdefault(orcid_id, {}).setdefault((fname + ' ' + lname), []).append(employment.get('department-name'))
                                request_log.write(tmpl % (datetime.isoformat(datetime.now()), counter, orcid_id))

                    except AttributeError:
                        pass
                except RequestException as e:
                    print (e.response.text)
                #print ('Processing record %d' % counter)
                counter += 1
            # pprint.pprint(persons)
        except RequestException as e:
            print (e.response.text)
        #time.sleep(3)
    return persons

# Start program run here
# define log file
import os
from datetime import datetime
if not os.path.exists('output'):
	os.makedirs('output')
    
# log 
request_log = open('output/orcid.log', 'w')
request_log.write('Timestamp\t\t\tstep\tORCID\n')

In [12]:
print (ringgoldid)
print (api.search_public('6197' )['orcid-search-results']['num-found'])
print (api.search_public('%s' % (ringgoldid))['orcid-search-results']['num-found'])
print (api.search_public('University of Oklahoma' )['orcid-search-results']['num-found'])
print (api.search_public('Oklahoma University')['orcid-search-results']['num-found'])
print (api.search_public('6197' 'University of Oklahoma' )['orcid-search-results']['num-found'])
print (api.search_public('6197' 'Oklahoma University' )['orcid-search-results']['num-found'])
print (api.search_public('University of Oklahoma' 'Oklahoma University')['orcid-search-results']['num-found'])
print (api.search_public('6197' 'University of Oklahoma' 'Oklahoma University')['orcid-search-results']['num-found'])
print (api.search_public('Oklahoma')['orcid-search-results']['num-found'])

6187
554
919
506126
316082
468671
316015
506116
506116
1066


In [13]:
print(api.search_public('University of Oklahoma')


SyntaxError: unexpected EOF while parsing (<ipython-input-13-bff000565714>, line 1)

In [ ]:
api.search_public?

In [ ]:
tester = search_persons ('6187', 'Oklahoma University')

In [ ]:
len (tester)

In [ ]:
tester.items()

### Working on csv output of results

In [ ]:
import csv

In [ ]:
for key1, value1 in tester.items():
    for key2, value2 in value1.items():
        print (key1)
        print (key2)
        print (value2)
    

In [ ]:
keys1 = tester.keys()

In [ ]:
keys1

In [ ]:
keys2 = tester.values

In [ ]:
c = tester.items()

In [ ]:
for c in tester.values():
    print (c)

In [ ]:
with open ('output/ou_orcid.csv', 'w', newline='') as fileout:
    peoplewriter = csv.writer(fileout, delimiter=',')
    for key1, value1 in tester.items():
        for key2, value2 in value1.items():
            row = (str(key1), str(key2), value2)
            peoplewriter.writerow(row)
fileout.close()

### Methods for recursively moving through a nested dictionary

In [ ]:
def myprint(d):
    stack = list(d.items())
    while stack:
        k, v = stack.pop()
        if isinstance(v, dict):
            stack.extend(v.items())
        else:
            print("%s: %s" % (k, v))

In [ ]:
myprint(tester)

In [ ]:
tester

In [ ]:
d = {
'key_1': 'value_1',
'key_2': {'key_21': [(2100, 2101), (2110, 2111)],
      'key_22': ['l1', 'l2'],
      'key_23': {'key_231': 'v'},
      'key_24': {'key_241': 502,
             'key_242': [(5, 0), (7, 0)],
             'key_243': {'key_2431': [0, 0],
                 'key_2432': 504,
                 'key_2433': [(11451, 0), (11452, 0)]
                },
             'key_244': {'key_2441': {'key_24411': {'key_244111': 'v_24411',
                                'key_244112': [(5549, 0)]
                               },
                          'key_24412':'v_24412'
                         },
                 'key_2441': ['ll1', 'll2']
                }
            },
     }
}

In [ ]:
myprint(d)

In [ ]:
def recurse(d, keys=()):
    if type(d) == dict:
         for k in d:
            for rv in recurse(d[k], keys + (k, )):
                yield rv
    else:
        yield (keys, d)
        
for compound_key, val in recurse(d):
    print ('{}: {}'.format(compound_key, val))

In [ ]:
for ids, val in recurse(tester):
    # print (ids, val)
    print ('\n')
    print ([ j for j in ids] +  [k for k in val])

In [ ]:
ids[0]

In [ ]:
for j in ids:
    print (j)

In [ ]:
from datetime import datetime


In [ ]:
datetime.isoformat(datetime.now())

In [ ]:
datetime.isoformat?